In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import newton

In [2]:
df = pd.read_csv('../data/occupation_shares.csv')
df = df.iloc[:-1]
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
df = df.set_index('MET2013')

In [12]:
# # Convert DataFrame to dictionary of vectors
# vectors = df.to_dict('list')

# # Normalize each vector
# for key in vectors:
#     vectors[key] = np.array(vectors[key]) / np.linalg.norm(vectors[key])

# def func(rho, column, target):
#     return np.sum(column ** (1 / (1 - rho))) - target

# def solve_rho(column, target):
#     rho_initial_guess = 0.5
#     rho_solution = newton(func, rho_initial_guess, args=(column, target))
#     return rho_solution

# # Remove the last key-value pair from vectors
# last_key = list(vectors.keys())[-1]
# last_column = vectors.pop(last_key)

# # Apply the solve_rho function to each vector
# rhos = {key: solve_rho(column, sum(column)) for key, column in vectors.items()}

In [10]:
def get_shifter(pi_c, pi_c_prime, rho_k=0.5):
    return (pi_c / pi_c_prime)**(1 - rho_k)

# Assuming df is your DataFrame
df_new = df.copy()

pi_c_prime = df.iloc[0, 0]
df_new = df.applymap(lambda x: get_shifter(x, pi_c_prime))
df_new.to_csv('shifter_estimates.csv')

/tmp/ipykernel_23317/3569516128.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_new = df.applymap(lambda x: get_shifter(x, pi_c_prime))
